In [100]:
import requests
import json
import nltk
import re
import pandas as pd
from datetime import datetime, timedelta, timezone

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\c24085394\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [102]:
API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "STU38746G38B7RB46GBER"

headers = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY
}


query_text = "what is happening in los angeles?"

payload = {
    "query_text": query_text,
    "result_size": 100,  
    "include_highlights": False,
    "include_smart_tags": False
}

try:
    response = requests.post(API_URL, headers=headers, data=json.dumps(payload), timeout=15)
    response.raise_for_status()
    json_response = response.json()
except requests.exceptions.RequestException as e:
    print(f"❌ Request failed: {e}")
    json_response = None

if not json_response or 'results' not in json_response:
    raise ValueError("❌ No results returned from the API.")



In [113]:
json_response['results'][0]

{'id': 'DL-596599c6f2d722d44f6706f8ffed8844',
 'title': 'Los Angeles Consulting Group June Events 2025',
 'url': 'https://missionmatters.com/los-angeles-consulting-group-june-events-2025',
 'summary': 'Adam Torres and Kathleen Tepley, Founder of Los Angeles Consulting Group, discuss upcoming events. LACG hosts two different business development advisory practices, both managed by Kathleen. It’s one of our monthly episodes where we bring Kathleen Epley on the line.',
 'score': 104.8759,
 'timestamp': '2025-06-03T14:57:05+00:00'}

In [114]:
df = pd.json_normalize(json_response['results'])

df.head()

,id,title,url,summary,score,timestamp
0,DL-596599c6f2d722d44f6706f8ffed8844,Los Angeles Consulting Group June Events 2025,https://missionmatters.com/los-angeles-consult...,"Adam Torres and Kathleen Tepley, Founder of Lo...",104.87590,2025-06-03T14:57:05+00:00
1,DL-c5e2f9b702ab3cb72dbc648e224d8293,Big Bear eaglet update: Sunny returns home fro...,https://www.nbclosangeles.com/news/local/big-b...,Sunny flapped her wings back home at around 11...,100.84105,2025-06-03T12:51:59
2,DL-d640b07f53c4a166f41599499cb31e74,European stocks mixed as trade tensions and U....,https://www.nbclosangeles.com/news/business/mo...,Tesla selloff Stream Los Angeles News for free...,97.43178,2025-06-05T23:07:47
3,DL-eed9e602d916f1d088aee98f614873d0,Cinespia’s big July line-up opens with ‘Top Gun’,https://www.nbclosangeles.com/the-scene/cinesp...,"""The Addams Family"" screens July 26 $24.11 and...",91.50728,2025-06-03T13:48:24
4,DL-ae117e8454b80d830cf8760d6d0119b3,Los Angeles immigration raid protests update,https://www.publicradiotulsa.org/2025-06-08/lo...,SCOTT DETROW: We're going to hear one more voi...,84.96997,2025-06-08T17:58:36


In [115]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\.\S+", "", text)
    text = text.lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [116]:
sia = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    """
    Returns:
      - neg, neu, pos, compound (-1 to +1)
    """
    return sia.polarity_scores(text)

In [118]:


df['published_date'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.date

df['clean_summary'] = df['summary'].apply(clean_text)
df['sentiment'] = df['clean_summary'].apply(get_sentiment_scores)

df['sent_neg'] = df['sentiment'].apply(lambda d: d['neg'])
df['sent_neu'] = df['sentiment'].apply(lambda d: d['neu'])
df['sent_pos'] = df['sentiment'].apply(lambda d: d['pos'])
df['sent_compound'] = df['sentiment'].apply(lambda d: d['compound'])

# Top/bottom sentiment summaries
print("\n😀 Most Positive Summary:")
print(df[['summary', 'sent_compound']].sort_values(by='sent_compound', ascending=False).head(1))

print("\n😞 Most Negative Summary:")
print(df[['summary', 'sent_compound']].sort_values(by='sent_compound').head(1))



😀 Most Positive Summary:
                                              summary  sent_compound
37  Los Angeles County celebrates Pride Month with...         0.9666

😞 Most Negative Summary:
                                             summary  sent_compound
8  Police say two people were shot dead on a busy...        -0.9186


In [119]:
today = datetime.now(timezone.utc).date()
seven_days_ago = today - timedelta(days=6)
recent_df = df[(df['published_date'] >= seven_days_ago) & (df['published_date'] <= today)]

if recent_df.empty:
    raise ValueError("❗ No news results from the past 7 days.")

In [120]:
print("\n📅 Daily Combined News Summary:\n")

for date in sorted(df['published_date'].dropna().unique()):
    print(f"\n📅 Date: {date}")
    
    # Get all summaries for the day
    daily_summaries = df[df['published_date'] == date]['summary']
    
    # Combine all summaries into one big text
    combined_summary = " ".join(daily_summaries)
    

    max_length = 1000  
    if len(combined_summary) > max_length:
        combined_summary = combined_summary[:max_length] + "..."
    
    print(f"📝 Combined Summary of All News ({len(daily_summaries)} articles):")
    print(combined_summary)
    
    print("\n" + "=" * 100 + "\n")



📅 Daily Combined News Summary:


📅 Date: 2025-06-03
📝 Combined Summary of All News (6 articles):
Adam Torres and Kathleen Tepley, Founder of Los Angeles Consulting Group, discuss upcoming events. LACG hosts two different business development advisory practices, both managed by Kathleen. It’s one of our monthly episodes where we bring Kathleen Epley on the line. The Duchess of Sussex will be honoured at the Natural History Museums of Los Angeles County. The glitzy event will imitate "an enchanted garden at night" with immersive exhibits, live music, and dinner served. Immediate pressure on trade and costs Since the beginning of May, the ports of Los Angeles and Long Beach have seen their activity fall by 35% over a year, according to Le Figaro. Average tariffs in the United States have risen from 2.5% to 24%, with surcharges of up to 145% on some Chinese products. This trade war by Donald Trump is causing companies to shrink. The city and county public library systems host the biggest 